<a href="https://colab.research.google.com/github/MK316/Spring2024/blob/main/DLTESOL/synonym_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synonym matching application

## [1] Original code from Megan

In [1]:
# Import libraries
from ipywidgets import Text, Button, VBox

# Define questions and answers
questions = ["colorful", "worried", "excited", "showing", "respected", "doubters", "success" ]
answers = ["vibrant", "anxious", "thrilled", "depicting", "admired", "skeptics", "achievement"]

# Initialize variables
score = 0
current_question = 0

# Define functions
def display_question():
    global current_question
    question_widget = Text(description=questions[current_question])
    check_button = Button(description="Check Answer")

    def check_answer(b):
        global current_question, score
        if question_widget.value.lower() == answers[current_question].lower():
            score += 1
            print("Correct!")
        else:
            print("Incorrect. The correct answer is", answers[current_question])
        current_question += 1
        if current_question < len(questions):
            display_question()
        else:
            print("Check how you did. Your score is", score, "out of", len(questions))

    check_button.on_click(check_answer)

    display(VBox([question_widget, check_button]))

# Let's match synonyms
display_question()

Correct!


# Miran versioin

In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.2 MB/

# this works!

In [38]:
import gradio as gr
import random

# Define questions and answers
questions = ["colorful", "worried", "excited", "showing", "respected", "doubters", "success"]
answers = ["vibrant", "anxious", "thrilled", "depicting", "admired", "skeptics", "achievement"]

answered_questions = set()  # Track which questions have been answered

def get_options(question):
    index = questions.index(question)
    correct_answer = answers[index]
    options = set([correct_answer])
    while len(options) < 4:
        options.add(random.choice(answers))
    options = list(options)
    random.shuffle(options)
    return ", ".join(options), correct_answer, index

def check_answer(user_input, correct_answer, score, question_index):
    if question_index in answered_questions:
        return f"You have already answered this question. Your score remains {score}.", score
    if user_input.strip().lower() == correct_answer.lower():
        score += 1
        feedback = f"Correct! Your score is now {score}."
    else:
        feedback = f"Incorrect. The correct answer was '{correct_answer}'. Your score remains {score}."
    answered_questions.add(question_index)
    return feedback, score

with gr.Blocks() as app:
    with gr.Row():
        question_dropdown = gr.Dropdown(choices=questions, label="Select a question", value=questions[0])
        show_options_button = gr.Button("Show me options")
    options_output = gr.Textbox(label="Options", interactive=False)
    correct_answer_store = gr.State()
    question_index_store = gr.State()
    user_input = gr.Textbox(label="Type your answer here", visible=True)
    submit_button = gr.Button("Submit Answer", visible=True)
    feedback_label = gr.Label()
    score_store = gr.State(value=0)
    complete_button = gr.Button("Complete and Show Total Score", visible=True)

    def show_options(question):
        options, correct, index = get_options(question)
        options_output.value = options
        correct_answer_store.value = correct
        question_index_store.value = index
        return options, correct, index

    show_options_button.click(
        fn=show_options,
        inputs=question_dropdown,
        outputs=[options_output, correct_answer_store, question_index_store]
    )

    submit_button.click(
        fn=lambda user_input, correct_answer_store=correct_answer_store, score_store=score_store, question_index_store=question_index_store: submit_response(user_input, correct_answer_store.value, score_store.value, question_index_store.value),
        inputs=user_input,
        outputs=[feedback_label, score_store]
    )

    def submit_response(user_input, correct_answer, score, question_index):
        feedback, updated_score = check_answer(user_input, correct_answer, score, question_index)
        score_store.value = updated_score  # Ensure the score is updated in the state
        return feedback, updated_score

    complete_button.click(
        fn=lambda score_store=score_store: f"Session Complete. Your final score is {score_store.value}.",
        inputs=[],
        outputs=feedback_label
    )

app.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://27062540eafa883aa8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Miran's message: This is the final code. You don't need requirements.txt file for this application because it only uses 'gradio' which is set when you create a Space.

In [47]:
import gradio as gr
import random

# Define questions and answers
questions = ["colorful", "worried", "excited", "showing", "respected", "doubters", "success"]
answers = ["vibrant", "anxious", "thrilled", "depicting", "admired", "skeptics", "achievement"]

answered_questions = set()  # Track which questions have been answered

def get_options(question):
    index = questions.index(question)
    correct_answer = answers[index]
    options = set([correct_answer])
    while len(options) < 4:
        options.add(random.choice(answers))
    options = list(options)
    random.shuffle(options)
    return ", ".join(options), correct_answer, index

def check_answer(user_input, correct_answer, question_index):
    if question_index in answered_questions:
        return "You have already answered this question.", 0
    if user_input.strip().lower() == correct_answer.lower():
        feedback = "Correct!"
        score = 1
    else:
        feedback = f"Incorrect. The correct answer was '{correct_answer}'."
        score = 0
    answered_questions.add(question_index)
    return feedback, score

with gr.Blocks() as app:
    with gr.Row():
        question_dropdown = gr.Dropdown(choices=questions, label="Select a question")
        show_options_button = gr.Button("Show me options")
    options_output = gr.Textbox(label="Options", interactive=False)
    correct_answer_store = gr.State()
    question_index_store = gr.State()
    user_input = gr.Textbox(label="Type your answer here", visible=True)
    submit_button = gr.Button("Submit Answer", visible=True)
    feedback_label = gr.Label()
    score_store = gr.State(value=0)
    complete_button = gr.Button("Complete and Show Total Score", visible=True)

    def show_options(question):
        options, correct, index = get_options(question)
        options_output.value = options
        correct_answer_store.value = correct
        question_index_store.value = index
        return options, correct, index

    show_options_button.click(
        fn=show_options,
        inputs=[question_dropdown],
        outputs=[options_output, correct_answer_store, question_index_store]
    )

    submit_button.click(
        fn=lambda user_input, correct_answer_store=correct_answer_store, question_index_store=question_index_store: submit_response(user_input, correct_answer_store.value, question_index_store.value),
        inputs=user_input,
        outputs=[feedback_label]
    )

    def submit_response(user_input, correct_answer, question_index):
        feedback, score = check_answer(user_input, correct_answer, question_index)
        score_store.value += score  # Accumulate the score in the state without showing it immediately
        return feedback

    complete_button.click(
        fn=lambda score_store=score_store: f"Session Complete. Your final score is {score_store.value}.",
        inputs=[],
        outputs=feedback_label
    )

app.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f1c40af84b01f64379.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
